<!---
###############################################################################
#
# Copyright (c) 2019, Xilinx
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# * Redistributions of source code must retain the above copyright notice, this
#   list of conditions and the following disclaimer.
#
# * Redistributions in binary form must reproduce the above copyright notice,
#   this list of conditions and the following disclaimer in the documentation
#   and/or other materials provided with the distribution.
#
# * Neither the name of the copyright holder nor the names of its
#   contributors may be used to endorse or promote products derived from
#   this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# Author: Izhar Shaikh <izhar.ameer.shaikh@xilinx.com>
#
###############################################################################
-->
# Suspend-Resume

This demonstrates suspend-resume power managemnent feature, with a RTC device used as a wakeup source.

1. [Introduction](#introduction)
2. [Linux Suspend-Resume](#linux-suspend-resume)
3. [Linux Suspend-Resume Demo](#linux-suspend-resume-demo)
4. [References](#xlnx-pm-wiki)

---

## Introduction <a name="introduction"></a>

The Linux kernel image binary included in Xilinx Petalinux Pre-built BSP for a Versal platform evaluation board
includes the feature which allows suspending (and subsequently resuming) the state of the system
to (or from) a low power state.

If you're not using Petalinux pre-built images and compiling using your own config,
following kernel configurations are required:

- Power management options
     - \[\*\] Suspend to RAM and standby
	 - \[\*\] User space wakeup sources interface interface
	 - \[\*\] Device power management core functionality
- Device Drivers
	 - SoC (System on Chip)-specific drivers
- Firmware Drivers
	 - Zynq MPSoC Firmware Drivers
	 - Enable Xilinx Zynq MPSoC firmware interface

**Note**: Any device can prevent the kernel from suspending.

---

## Linux Suspend-Resume <a name="linux-suspend-resume"></a>

The kernel supports up to four system sleep states generically, although three
of them depend on the platform support code to implement the low-level details
for each state.

The states are represented by strings that can be read or written to the
`/sys/power/state` file.  Those strings may be `mem`, `standby`, `freeze` and
`disk`, where the last three always represent *Power-On Suspend* (if supported),
*Suspend-To-Idle* and *hibernation* (Suspend-To-Disk), respectively.

In [ ]:
cat /sys/power/state

The meaning of the `mem` string is controlled by the `/sys/power/mem_sleep` file.
It contains strings representing the available modes of system suspend that may
be triggered by writing `mem` to `/sys/power/state`.  These modes are "s2idle"
(Suspend-To-Idle), "shallow" (Power-On Suspend) and "deep" (Suspend-To-RAM).

The "s2idle" mode is always available, while the other ones are only available
if supported by the platform (if not supported, the strings representing them
are not present in `/sys/power/mem_sleep`).  The string representing the suspend
mode to be used subsequently is enclosed in square brackets.  Writing one of
the other strings present in `/sys/power/mem_sleep` to it causes the suspend mode
to be used subsequently to change to the one represented by that string.

In [ ]:
cat /sys/power/mem_sleep

The default suspend mode (ie. the one to be used without writing anything into
`/sys/power/mem_sleep`) is either `deep` (if Suspend-To-RAM is supported) or
`s2idle`, but it can be overridden by the value of the `"mem_sleep_default"`
parameter in the kernel command line.

For the purpose of the demo, we'll be using `deep` since we'll be suspending to RAM.

This state, offers significant power savings as everything in the
system is put into a low-power state, except for memory, which will be placed
into the self-refresh mode to retain its contents.  All of the steps carried out
when entering Power-On Suspend are also carried out during transitions to STR (Suspend-To-RAM).
Additional operations may take place depending on the platform capabilities.

System and device state is saved and kept in memory.  All devices are suspended
and put into low-power states.  In many cases, all peripheral buses lose power
when entering STR, so devices must be able to handle the transition back to the
"on" state.

Some drivers can manage hardware wakeup events, which make the system leave the low-power state.

The kernel resumes from the suspend mode when a wake-up event occurs.

RTC, UART and GPIO devices can be used as wakeup sources.
However, for the purpose of this demo, we'll only be focusing on RTC as a wakeup source.
For more information on this, please refer to Versal Software Developers Guide.

---

## Linux Suspend-Resume Demo <a name="linux-suspend-resume-demo"></a>

To run this demo, following requirement **must** be met:
  - System must be booted from a Non-JTAG boot mode.

This demo assumes that you're using the boot PDI (BOOT.BIN) from the pre-built artifacts of Petalinux BSP for the eval board, and the system is booted from a Non-JTAG boot mode.

For the purpose of this demo, we'll be using RTC as a wakeup source.

Please note that if RTC is enabled as a wake-up source, the kernel will resume from the suspend mode when the RTC
timer expires. It is also to be noted that RTC wake-up source is enabled by default.

lets run the demo:

Following sets up the timeout value of RTC as 15 seconds and triggers system suspend.
Once this value expires, system will resume from suspend state.

In [ ]:
!echo +15 > /sys/class/rtc/rtc0/wakealarm && echo mem > /sys/power/state

## References <a name="xlnx-pm-wiki"></a>
 - [Xilinx PM Wiki Page for Zynq UltraScale＋](https://xilinx-wiki.atlassian.net/wiki/spaces/A/pages/18842232)
 - [Device Power Management Basics](https://www.kernel.org/doc/html/v4.14/driver-api/pm/devices.html)
 - [Suspend and Hibernate](https://wiki.archlinux.org/index.php/Power_management/Suspend_and_hibernate)